In [ ]:
!pip -q install langchain tiktoken chromadb pypdf transformers InstructorEmbedding
!pip -q install accelerate bitsandbytes sentencepiece Xformers

In [ ]:
import torch 
import transformers
from transformers import LlamaTokenizer, LlamaForCasualLM, GenerationConfig, pipeline

tokenizer = LlamaTokenizer.from_pretrained("TheBloke/wizardLM-7b-HF")

model = LlamaToeknizer.from_pretrained("TheBloke/wizardLM-7b-HF", 
                                        load_in_8bit=True,
                                        device_map='auto', 
                                        torch_dtype=torch.float16,
                                        low_cpu_mem_usage=True
                                        )

In [ ]:
from langchain.llms import HuggingFacePipeline

pipe = pipeline(
        "text-generation", 
        model=model,
        tokenizer=tokenizer,
        max_length=1024,
        temperature=0,
        top_p=.95,
        repetition_penalty=1.15
      )

local_llm = HuggingFacePipeline(pipeline=pipe)

# LangChain multi-doc retriever with ChromaDB

## Setting up LangChain


In [ ]:
import os

In [ ]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

## Load multiple documents and process


In [ ]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('./assets/ncvs_documents', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

In [ ]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

## HuggingFace Instructor Embeddings


In [ ]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})

## Create the Database


In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

## Make a retriever


In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

## Make a chain


In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=local_llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

## Cite sources


In [ ]:
import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])